In [1]:
import os
import torch
import numpy as np
import pandas as pd
from CellClass import CNN
from natsort import natsorted
import matplotlib.pyplot as plt
import CellClass.CNN.dataset as D
import CellClass.CNN.training as T
from torch.utils.data import DataLoader

In [2]:
save_dir = "/home/simon_g/src/MICCAI/trained_models"

setups = os.listdir(save_dir)

models = []
for setup in setups:
    files = [os.path.join(save_dir, setup, x) for x in os.listdir(os.path.join(save_dir, setup))]
    if any([x for x in files if ".png" in x]):
        models.extend([x for x in files if ".pt" in x])
    
losses = []
for model in models:
    dict_mod = torch.load(model)
    losses.append({"model": model, "loss": dict_mod["validation_loss"]})

In [3]:
model_df = pd.DataFrame(losses)

In [4]:
sorted = model_df.sort_values("loss")
best = sorted.iloc[0,:]
print(best)
print(best.model)

model    /home/simon_g/src/MICCAI/trained_models/202206...
loss                                              0.000142
Name: 13, dtype: object
/home/simon_g/src/MICCAI/trained_models/20220621_185131/CNN_Model_20220621_190936917.pt


In [5]:
best_setup = best.model.split("/")[-2]
with open(os.path.join(save_dir, best_setup, "training_parameters.txt"), "r") as fin:
    lines = fin.readlines()

    lines = [x.strip().split(":") for x in lines]
    lines = np.array(lines)
    
setup_df = pd.DataFrame(lines.T[1,:], index=lines.T[0,:], columns=[0]).T
setup_df = setup_df.loc[:,~setup_df.columns.duplicated()].copy()

In [6]:
try:
    layers = np.array(setup_df.layers.item().strip().replace("[", "").replace("]", "").split(",")).astype(int)
    in_shape = np.array(setup_df.in_shape.item().strip().replace("[", "").replace("]", "").split(",")).astype(int)

except:
    layers = np.array([3, 16, 64, 128, 256])
    in_shape = np.array([128, 128])

In [7]:
best = torch.load(best.model, map_location="cpu")
model = CNN.ClassificationCNN(layers=layers, in_shape=in_shape)
model.load_state_dict(best["model_state_dict"])

<All keys matched successfully>

In [8]:
patches_dir = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/MYCN_SpikeIn/results/patches"
dilutions = np.unique([x.split("_")[0] for x in os.listdir(patches_dir)])
print(dilutions)

['S11' 'S12' 'S19' 'S1b' 'S2' 'S20' 'S29' 'S3' 'S30' 'S31' 'S32' 'S33'
 'S34' 'S4' 'S6' 'S7' 'S8']


In [9]:
dils = []
for dilution in dilutions: 
    files = [x for x in os.listdir(patches_dir) if f"{dilution}_" in x]
    dils.append({"series": dilution, "files": files})
    print(dilution, len(files))

S11 113
S12 123
S19 597
S1b 40
S2 45
S20 116
S29 597
S3 37
S30 101
S31 20
S32 116
S33 94
S34 96
S4 28
S6 30
S7 36
S8 43


In [10]:
original_dilutions = [{
    "S1b": 95,
    "S2": 75,
    "S3": 50,
    "S4": 100,
    "S6": 0,
    "S7": 90,
    "S8": 25,
    "S11": 99,
    "S12": 0,
    "S19": 100,
    "S20": 99,
    "S29": 0,
    "S30": 95,
    "S31": 50,
    "S32": 90,
    "S33": 25,
    "S34": 75,
}]

In [11]:
dilutions = natsorted(dilutions)
results = {}
for dil in dilutions:
    dilution = D.load_dilution(patches_dir, f"{dil}_", n=10, verbose=False)
    dilution_loader = DataLoader(dilution, 128)
    ims, labels, percentage = T.predict_dilution(model, dilution_loader)
    
    # fig, axs = plt.subplots(5, 5, figsize=(10,10))
    # for ax, im, l in zip(axs.ravel(), ims[:25], labels[:25]):
    #     ax.imshow(im)
    #     ax.set_title(l)
    #     ax.set_xticks([])
    #     ax.set_yticks([])
    # plt.show()
    results[dil] = percentage

    print(f"{dil} [{len(dilution)} Patches] with predicted dilution of: {percentage}")
    

S1b [2551 Patches] with predicted dilution of: 69.74
S2 [2679 Patches] with predicted dilution of: 81.04
S3 [3007 Patches] with predicted dilution of: 46.69
S4 [5592 Patches] with predicted dilution of: 95.98
S6 [4846 Patches] with predicted dilution of: 0.21
S7 [3156 Patches] with predicted dilution of: 88.72
S8 [2814 Patches] with predicted dilution of: 21.71
S11 [3607 Patches] with predicted dilution of: 90.27
S12 [1975 Patches] with predicted dilution of: 0.91
S19 [1245 Patches] with predicted dilution of: 99.52
S20 [2087 Patches] with predicted dilution of: 95.83
S29 [2932 Patches] with predicted dilution of: 0.44
S30 [3938 Patches] with predicted dilution of: 95.1
S31 [4039 Patches] with predicted dilution of: 51.45
S32 [2430 Patches] with predicted dilution of: 93.13
S33 [2536 Patches] with predicted dilution of: 9.74
S34 [3314 Patches] with predicted dilution of: 77.04


In [12]:
df_target = pd.DataFrame(original_dilutions)
df_prediction = pd.DataFrame([results])


merged = pd.concat((df_target, df_prediction), axis=0)

merged.head()

,S1b,S2,S3,S4,S6,S7,S8,S11,S12,S19,S20,S29,S30,S31,S32,S33,S34
0,95.00,75.00,50.00,100.00,0.00,90.00,25.00,99.00,0.00,100.00,99.00,0.00,95.0,50.00,90.00,25.00,75.00
0,69.74,81.04,46.69,95.98,0.21,88.72,21.71,90.27,0.91,99.52,95.83,0.44,95.1,51.45,93.13,9.74,77.04


In [15]:
l = {}
for i,j in merged.items():
   l[i] = j.iloc[1]-j.iloc[0]
   
merged = pd.concat((merged, pd.DataFrame([l])), axis=0)

In [16]:
merged.index = ["original", "prediction", "diff"]
merged.head()

,S1b,S2,S3,S4,S6,S7,S8,S11,S12,S19,S20,S29,S30,S31,S32,S33,S34
original,95.00,75.00,50.00,100.00,0.00,90.00,25.00,99.00,0.00,100.00,99.00,0.00,95.0,50.00,90.00,25.00,75.00
prediction,69.74,81.04,46.69,95.98,0.21,88.72,21.71,90.27,0.91,99.52,95.83,0.44,95.1,51.45,93.13,9.74,77.04
diff,-25.26,6.04,-3.31,-4.02,0.21,-1.28,-3.29,-8.73,0.91,-0.48,-3.17,0.44,0.1,1.45,3.13,-15.26,2.04
